In [ ]:
# %cd ./work
# %pwd

In [ ]:
%run "../catalog_common.py"

In [ ]:
# import sys
# sys.path.insert(0,'c:/MyDocs/OpenFF/src/openFF-catalog/')
# import catalog_common as cc

In [ ]:
#preamble to analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import gca, mpl

%matplotlib inline
import seaborn as sns
import matplotlib.ticker
from IPython.display import Markdown as md
from IPython.display import HTML, display
from time import sleep

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt
opt.order = []  # no sorting

In [ ]:
alldf = pd.read_csv('state.csv',low_memory=False)
alldf.date = pd.to_datetime(alldf.date)
statename = alldf.bgStateName.iloc[0]

In [ ]:
ID_header(statename.title(), subtitle='Open-FF State Summary',incl_links=True,
          link_up_level=True)
set_page_param()

In [ ]:
def xlate_val(n):
    if n==0:
        return ''
    if n<1000:
        return round_sig(n,1)
    x = round_sig(n,1)
    return x[0]+ 'k'

def make_annot(gb):
    annot = gb.copy()
    annot.UploadKey = annot.UploadKey.map(lambda x: xlate_val(x))
    #print(annot)
    piv = annot.pivot(index='County',columns='year',values='UploadKey')
    piv.fillna('',inplace=True)
    #print(piv)
    return piv
    
def CountyMap(df):
    start_loc = get_state_center(statename)
    #print(statename,start_loc)
    cond = (df.loc_within_state=='YES')&(df.loc_within_county=='YES')
    if cond.sum()==0:  # no valid fracks for this state
        display(md('## No mappable fracks for this state!'))
        display(md(f'Any data in this state set may be labeled incorrectly as {statename}'))
        return
    gb = df[cond].groupby(['bgStateName','bgCountyName',
                                                   'UploadKey'],as_index=False)['bgCAS'].count()
    gb = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['UploadKey'].count().rename({'bgStateName':'StateName',
                                                                                                'bgCountyName':'CountyName',
                                                                                                'UploadKey':'value'},
                                                                                                axis=1)
    zoom = 6
    if statename in ['texas','california']:
        zoom = 5
    if statename in ['alaska']:
        zoom = 4
        
    create_county_choropleth(gb,plotlog=True,custom_scale= [0,1,2,3,4,5],
                             start_loc=start_loc, # center of state's data
                             legend_name='Number of FracFocus disclosures',
                             start_zoom=zoom,fields=['StateName','CountyName','orig_value'],
                             aliases=['State: ','County: ','Number Fracking disclosures: '])

def CountyCntTable(df):
    # first, make the general searchable table
    gb = df.groupby(['bgCountyName','UploadKey'],as_index=False)['date'].first()
    gb['year'] = gb.date.dt.year.astype('str')
    gb1 = gb.groupby(['bgCountyName'],as_index=False)['UploadKey'].count().rename({'UploadKey':'disclosure_count'},
                                                                                  axis=1)
    gb2 = gb1.copy()
    gbop = df.groupby('bgCountyName')['bgOperatorName'].agg(lambda x:x.value_counts().index[0:4]).reset_index()
    gbop.bgOperatorName = gbop.bgOperatorName.map(lambda x: xlate_to_str(x,'; ',sort=False))
    gbop = gbop.rename({'bgOperatorName':'Top Operators'},axis=1)
    gb2 = pd.merge(gb2,gbop,on='bgCountyName',how='left')

    gbprop = df[df.bgCAS=='proprietary'].groupby('bgCountyName',as_index=False)['bgCAS'].count()
#     gbprop.bgCAS.fillna(0,inplace=True)
    gbprop = gbprop.rename({'bgCAS':'Trade Secret records'},axis=1)
    gb2 = pd.merge(gb2,gbprop,on='bgCountyName',how='left')
    gb2['Trade Secret records'].fillna(0,inplace=True)
    
    gbtbwv = df.groupby(['bgCountyName','UploadKey'],as_index=False)['TotalBaseWaterVolume'].first()
    gbtbwv = gbtbwv.groupby('bgCountyName',as_index=False)['TotalBaseWaterVolume'].sum().rename({'TotalBaseWaterVolume':'tot_gallons_water'},
                                                                                                axis=1)
    gbtbwv.tot_gallons_water = gbtbwv.tot_gallons_water.map(lambda x: round_sig(x,3))
    gb2 = pd.merge(gb2,gbtbwv,on='bgCountyName',how='left')

    #print(gb2.head())
    gb2['County'] = '<center><h4>'+gb2.bgCountyName.str.title().map(lambda x: getCountyLink(x,statename,x))+'</h4></center>'
    gb2 = gb2.drop('bgCountyName',axis=1)
    iShow(gb2.sort_values('disclosure_count',ascending=False)[['County','disclosure_count',
                                                               'Trade Secret records','tot_gallons_water',
                                                               'Top Operators']].reset_index(drop=True),
         classes="display compact cell-border")
        
#     # Now make the heatmap
#     gb3 = gb.groupby(['bgCountyName','year'],as_index=False)['UploadKey'].count()
#     gb3 = gb3.rename({'bgCountyName':'County'},axis=1)
#     gb3 = pd.merge(gb3,gb1,left_on='County',right_on='bgCountyName',how='left')
#     gb3.County = gb3.County.str.title()

#     gb_annot = make_annot(gb3)
#     #print(gb_annot)
#     gb3.UploadKey = gb3.UploadKey/gb3.disclosure_count *100
#     piv = gb3.pivot(index='County',columns='year',values='UploadKey')
#     #piv = piv.reset_index()
#     #piv['County'] = '<center><h3>'+piv.CountyName.str.title().map(lambda x: getCountyLink(x,statename,x))+'</h3></center>'
#     #piv = piv.drop('CountyName',axis=1)
#     piv.fillna(0,inplace=True)

    
#     fig = plt.figure(figsize=(len(piv.columns)*.75,len(piv)/3+3))
#     ax = sns.heatmap(piv,cmap="Reds",annot=gb_annot,fmt='')
#     plt.ylabel(f'Counties',fontsize=14);
#     plt.xlabel(f'Year',fontsize=14);
#     plt.title(f"Percent Distribution of county's disclosures by year",fontsize=16)
#     #plt.title("Annotations are the number of disclosures (rounded)",fontsize=12)
#     ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
#     ax.set_yticklabels(ax.get_yticklabels(),rotation = 0,fontsize=14);
#     ax.xaxis.set_ticks_position('top')
#     plt.show()
#     display(md("     Annotations are the number of disclosures that year (rounded)"))

# Where are the fracking locations in this state?
This is not an exhaustive set of wells in these counties; it is only those wells for which the operating company submits a chemical disclosure to FracFocus.  In addition, this map omits disclosures for which location information is conflicting, such as the Latitude/Longitude values are outside of the reported county.

In [ ]:
CountyMap(alldf)

---
## County-based details

In [ ]:
CountyCntTable(alldf)

---
## Who are the Operators in this state?

In [ ]:
def make_water(row):
    s = str(round_sig(row.TotalBaseWaterVolume,3))
    s += '<br>'
    s += str(round_sig(row.TBWV90,3))
    return s

alldf['year'] = alldf.date.dt.year.astype(str)
gbOp = alldf.groupby(['UploadKey','bgCountyName','bgOperatorName','year'],as_index=False)['TotalBaseWaterVolume'].first()
gbOp.bgCountyName = gbOp.bgCountyName.str.title()

gbOp1 = gbOp.groupby('bgOperatorName',as_index=False)['UploadKey'].count().rename({'UploadKey':'num_fracks'},axis=1)

gbOp2 = gbOp.groupby(['bgOperatorName','year'],as_index=False)['UploadKey'].count()
gbOp2['year_cnt'] = gbOp2.year + '(' + gbOp2.UploadKey.astype(str) + ')'

gbOpY = gbOp2.groupby('bgOperatorName')['year_cnt'].apply(set).reset_index()
gbOpY['years'] = gbOpY.year_cnt.map(lambda x: xlate_to_str(x,sep='<br>'))

gbOp3 = gbOp.groupby(['bgOperatorName','bgCountyName'],as_index=False)['UploadKey'].count()
gbOp3['counties_cnt'] = gbOp3.bgCountyName + '(' + gbOp3.UploadKey.astype(str) + ')'

gbOp4 = gbOp3.groupby('bgOperatorName')['counties_cnt'].apply(set).reset_index()
gbOp4['counties'] = gbOp4.counties_cnt.map(lambda x: xlate_to_str(x,sep='<br>'))

# gbOp5 = df.groupby('bgOperatorName')['OperatorName'].agg(lambda x: x.value_counts().index[0])
gbOp5 = alldf.groupby('bgOperatorName')['OperatorName'].apply(set).reset_index()
gbOp5['names'] = gbOp5.OperatorName.map(lambda x: xlate_to_str(x,sep='<br>'))
# gbOp5.names = gbOp5.names + '<br>[' + gbOp5.bgOperatorName + ']'

gbOp6 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].median()
gbOp6.rename({'TotalBaseWaterVolume':'Water,\nmedian (gal)'},axis=1,inplace=True)
# gbOp7 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].agg(lambda x: np.percentile(x,90))
gbOp7 = gbOp.groupby('bgOperatorName',as_index=False)['TotalBaseWaterVolume'].max()
gbOp7.rename({'TotalBaseWaterVolume':'Water,\nmax (gal)'},axis=1,inplace=True)
mg = pd.merge(gbOp6,gbOp7,on='bgOperatorName')
# mg.fillna(0,inplace=True)
# mg['TBWV'] = mg.apply(lambda x: make_water(x),axis=1)
mg = pd.merge(mg,gbOp1,on='bgOperatorName')
mg = pd.merge(mg,gbOpY,on='bgOperatorName')
mg = pd.merge(mg,gbOp4[['bgOperatorName','counties']],on='bgOperatorName')
mg = pd.merge(mg,gbOp5,on='bgOperatorName').sort_values('num_fracks',ascending=False)

mg['link'] = mg.bgOperatorName.map(lambda x: getOpLink(x,x,up_level=True))
mg.names = mg.names + '<br>[' + mg.link + ']'

iShow(mg[['names','num_fracks','years','counties','Water,\nmedian (gal)','Water,\nmax (gal)']].reset_index(drop=True),
      maxBytes=0,columnDefs=[{"width": "150px", "targets": 0}])

In [ ]:
from matplotlib.offsetbox import AnchoredText


def proprietary_plot(df,plot_title='TEST',minyr=2011,maxyr=2021):
    df = df.copy()
    df['year'] = df.date.dt.year
    df = df[df.year<=maxyr]
    df = df[df.year>=minyr]
    prop = df.bgCAS=='proprietary'
    gb = df[prop].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numprop'},axis=1)
    gb1 = df[df.is_valid_cas].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numvalid'},axis=1)
    gb2 = df.groupby('UploadKey',as_index=False)['date'].first()
    mg = pd.merge(gb2,gb,on='UploadKey',how='left')
    mg = pd.merge(mg,gb1,on='UploadKey',how='left')
    mg.fillna(0,inplace=True) # there will be disclosures with 0 proprietary; need to fill
    mg['percProp'] = (mg.numprop / mg.numvalid) * 100

    mg['propCut'] = pd.cut(mg.percProp,right=False,bins=[0,0.0001,10,25,50,101],
                          labels=['no proprietary claims','up to 10% proprietary claims',
                                  'between 10 and 25% proprietary claims',
                                  'between 25 and 50% proprietary claims',
                                  'greater than 50% proprietary claims'])
    mg['year'] = mg.date.dt.year
    out = mg.drop(['date','UploadKey'],axis=1)
    t = out[out.numvalid>0].groupby(['year','propCut'],as_index=False)['numvalid'].count()
    sums = t.groupby('year',as_index=False)['numvalid'].sum().rename({'numvalid':'tot'},axis=1)
    t = pd.merge(t,sums,on='year',how='left')
    t['PercentProp'] = t.numvalid/t.tot *100

    piv = t.pivot(index='year', columns='propCut', values='PercentProp')

    ax = piv.plot.area(figsize=(12,7),ylim=(0,100),xlim=(minyr,maxyr),colormap='Reds')
    ax.set_title(f'Percentage of valid records that are Trade Secret claims at the disclosure level', fontsize=16)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], title='Disclosure Proprietary\nPercentage class\n',
              loc='upper left',bbox_to_anchor=(1, 1))
    ax.set_ylabel('Percentage of disclosures', fontsize=16)
    ax.set_xlabel('Year', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.suptitle(f'{plot_title}',fontsize=24)

    gb = df.groupby(['year','UploadKey'],as_index=False)['bgCAS'].count()
    gb = gb.groupby('year',as_index=False)['UploadKey'].count()#.rename({'UploadKey':'number of disclosures'},axis=1)
    s = 'Number of disclosures by year:\n\n'
    for i,row in gb.iterrows():
        s+= f'   {row.year}: {row.UploadKey:7,} \n'
    at2 = AnchoredText(s,
                       loc='lower left', prop=dict(size=10), frameon=False,
                       bbox_to_anchor=(1., 0.),
                       bbox_transform=ax.transAxes
                       )
    at2.patch.set_boxstyle("square,pad=0.")
    ax.add_artist(at2)

    
# test = 'pennsylvania'
# variable = 'bgStateName'
testtitle = statename.title() +': Trade Secret frequency'
if len(alldf.UploadKey.unique())>1000:
    # small-number states look silly with this plot
    proprietary_plot(alldf,testtitle,minyr=2011,maxyr=alldf.date.dt.year.max())


## School districts
|field| Description |
| :--: | -- |
|**NAME**| School District name |
|**num_FF_wells** | The number of wells within the district that have reported data in FracFocus. This covers the period from 2011 to the recent. |
|**num_all_wells** |  The number of all wells within the district that are recorded by the state-based data, and included non-fracked wells and may go back to the beginning of state-held record keeping - may decades.|
|**GEOID**| The national code used to identify this school district. | 

*Note that many districts will have wells, but no FracFocus wells.  Many will be older and non-fracked; others may not be recorded in FracFocus even though they are fracked.  Furthermore, we don't have state data for all US states yet; we have focused on those that are heavily represented in FracFocus.*


In [ ]:
wells_in_dist_fn = 'FFwells_in_school_districts.csv'
distFF = pd.read_csv(wells_in_dist_fn)
distFF = distFF[distFF.bgStateName==statename]
distFF.num_FF_wells.fillna(0,inplace=True)
iShow(distFF[['NAME','GEOID','num_FF_wells','num_all_wells']].reset_index(drop=True),
     classes="display compact cell-border")